# Creating a model and training it for generating and clustering music

### Downloading the dataset

In [ ]:
"""import kagglehub

# Download latest version
path = kagglehub.dataset_download("imsparsh/fma-free-music-archive-small-medium")

print("Path to dataset files:", path)"""

### Functions needed to transform the .mp3 soundfiles to .jpg images for the training set

In [1]:
import numpy as np
import librosa
import librosa.display
import random
from PIL import Image
import gc
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile as wav
from numpy.lib import stride_tricks
import os
# Reading the audio file and applying some transformations (trimming, padding...) to "clean" the sound file

def read_audio(conf, pathname, trim_long_data):
    y, sr = librosa.load(pathname, sr=conf.sampling_rate)
    # trim silence
    if 0 < len(y): # workaround: 0 length causes error
        y, _ = librosa.effects.trim(y) # trim, top_db=default(60)
    # make it unified length to conf.samples
    if len(y) > conf.samples: # long enough
        if trim_long_data:
            y = y[0:0+conf.samples]
    else: # pad blank
        padding = conf.samples - len(y)    # add padding at both ends
        offset = padding // 2
        y = np.pad(y, (offset, conf.samples - len(y) - offset), 'constant')
    return y

# Thanks to the librosa library, generating the mel-spectogram from the audio file

def audio_to_melspectrogram(conf, audio):
    spectrogram = librosa.feature.melspectrogram(y=audio, 
                                                 sr=conf.sampling_rate,
                                                 n_mels=conf.n_mels,
                                                 hop_length=conf.hop_length,
                                                 n_fft=conf.n_fft,
                                                 fmin=conf.fmin,
                                                 fmax=conf.fmax)
    spectrogram = librosa.power_to_db(spectrogram)
    spectrogram = spectrogram.astype(np.float32)
    return spectrogram

# Adding both previous function together

def read_as_melspectrogram(conf, pathname, trim_long_data, debug_display=False):
    x = read_audio(conf, pathname, trim_long_data)
    mels = audio_to_melspectrogram(conf, x)
    return mels

# A set of settings that you can adapt to fit your audio files (frequency, average duration, number of Fourier transforms...)

class conf:
    sampling_rate = 44100
    duration = 30
    samples = sampling_rate * duration
    n_mels = 256
    hop_length = 512
    n_fft = 2048  # can stay as 2048 or higher
    fmin = 20
    fmax = sampling_rate // 2


"""
def mono_to_color(X, mean=None, std=None, norm_max=None, norm_min=None, eps=1e-6):
    # Stack X as [X,X,X]
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    std = std or X.std()
    Xstd = (X - mean) / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Scale to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V
"""

def normalize_spectrogram(x, global_min=-100.00001, global_max=43.668285):
    return (x - global_min) / (global_max - global_min)

def rename_file(img_name):
    img_name = img_name.split("/")[-1]
    img_name = img_name[:-4]
    img_name += ".jpg"
    return img_name

def save_image_from_sound(img_path):
    filename = rename_file(img_path)
    x = read_as_melspectrogram(conf, img_path, trim_long_data=False, debug_display=True)
    #x_color = mono_to_color(x)
    x_norm=normalize_spectrogram(x)
    x_img = (x_norm * 255).astype(np.uint8)
    img = Image.fromarray(x_img, mode='L')
    filename = os.path.basename(rename_file(img_path))  # Only get "000005.jpg"
    save_dir = 'trainImages'
    img.save(os.path.join(save_dir, filename))
    del x
    gc.collect()

### Calculate the global min and max to use that for normalization and reconstruction of sound from generated images

In [3]:
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
import librosa

def calculate_global_min_max(full_path, conf):
    try:
        # Convert audio to mel spectrogram
        mel_spec = read_as_melspectrogram(conf, full_path, trim_long_data=False)
        # Return the min and max of the spectrogram
        return mel_spec.min(), mel_spec.max()
    except Exception as e:
        print(f"Error processing {full_path}: {e}")
        return None

def calculate_dataset_min_max():
    root_dir = '/home/rasmus/.cache/kagglehub/datasets/imsparsh/fma-free-music-archive-small-medium/versions/1/fma_medium'  # Change to your path
    all_mins = []
    all_maxs = []
    futures = []
    
    # List all mp3 files
    all_files = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for fn in filenames:
            if fn.endswith('.mp3'):
                all_files.append(os.path.join(dirpath, fn))
    
    total_files = len(all_files)
    print(f"Total files to process: {total_files}")
    
    with ThreadPoolExecutor() as executor:
        for full_path in all_files:
            futures.append(executor.submit(calculate_global_min_max, full_path, conf))
        
        # Track progress as files are completed
        for idx, future in enumerate(as_completed(futures)):
            if idx % 500 == 0 or idx == total_files - 1:  # Print progress every 500 files or on the last file
                print(f"Progress: {idx + 1}/{total_files} files processed")
                
            result = future.result()
            if result:
                all_mins.append(result[0])
                all_maxs.append(result[1])
    
    # Compute the overall min and max from all spectrograms
    global_min = np.min(all_mins)
    global_max = np.max(all_maxs)
    
    print(f"Global min: {global_min}, Global max: {global_max}")
    return global_min, global_max

calculate_dataset_min_max()

Total files to process: 25000
Progress: 1/25000 files processed
Progress: 501/25000 files processed


[src/libmpg123/parse.c:do_readahead():1083] warning: Cannot read next header, a one-frame stream? Duh...
/tmp/ipykernel_28282/1302790898.py:15: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(pathname, sr=conf.sampling_rate)
/home/rasmus/.local/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error processing /home/rasmus/.cache/kagglehub/datasets/imsparsh/fma-free-music-archive-small-medium/versions/1/fma_medium/fma_medium/065/065753.mp3: 
Progress: 1001/25000 files processed
Progress: 1501/25000 files processed
Progress: 2001/25000 files processed
Progress: 2501/25000 files processed


[src/libmpg123/layer3.c:INT123_do_layer3():1804] error: dequantization failed!


Progress: 3001/25000 files processed
Progress: 3501/25000 files processed


...

[src/libmpg123/parse.c:do_readahead():1083] warning: Cannot read next header, a one-frame stream? Duh...
/tmp/ipykernel_28282/1302790898.py:15: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(pathname, sr=conf.sampling_rate)
/home/rasmus/.local/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error processing /home/rasmus/.cache/kagglehub/datasets/imsparsh/fma-free-music-archive-small-medium/versions/1/fma_medium/fma_medium/126/126981.mp3: 
Progress: 24001/25000 files processed


[src/libmpg123/layer3.c:INT123_do_layer3():1844] error: dequantization failed!


Progress: 24501/25000 files processed
Progress: 25000/25000 files processed
Global min: -100.00000762939453, Global max: 43.66828536987305


(-100.00001, 43.668285)

### Convert each song in the dataset to an image and save it

In [2]:
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

def process_file(full_path):
    try:
        exists = os.path.isfile(rename_file(full_path))
        if not exists:
            save_image_from_sound(full_path)
    except Exception as e:
        print(f"Error processing {full_path}: {e}")
        return  #Continue processing other files even if one fails

def ParalellConvertImage():
    root_dir = '/home/rasmus/.cache/kagglehub/datasets/imsparsh/fma-free-music-archive-small-medium/versions/1/fma_medium' #Change to your path
    futures = []
    total_files = sum([len(files) for _, _, files in os.walk(root_dir) if any(file.endswith('.mp3') for file in files)])
    with ThreadPoolExecutor() as executor:
        for dirpath, dirnames, filenames in os.walk(root_dir):
            for fn in filenames:
                if fn.endswith('.mp3'):
                    full_path = os.path.join(dirpath, fn)
                    futures.append(executor.submit(process_file, full_path))
                    
        for idx, future in enumerate(as_completed(futures)):
            if idx%500 == 0:
                print(f"Progress: {idx + 1}/{total_files} files processed")

ParalellConvertImage()

Progress: 1/25000 files processed
Progress: 501/25000 files processed


[src/libmpg123/parse.c:do_readahead():1083] warning: Cannot read next header, a one-frame stream? Duh...
/tmp/ipykernel_27187/2012216425.py:15: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(pathname, sr=conf.sampling_rate)
/home/rasmus/.local/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error processing /home/rasmus/.cache/kagglehub/datasets/imsparsh/fma-free-music-archive-small-medium/versions/1/fma_medium/fma_medium/065/065753.mp3: 
Progress: 1001/25000 files processed
Progress: 1501/25000 files processed
Progress: 2001/25000 files processed
Progress: 2501/25000 files processed


[src/libmpg123/layer3.c:INT123_do_layer3():1804] error: dequantization failed!


Progress: 3001/25000 files processed
Progress: 3501/25000 files processed


...

Error processing /home/rasmus/.cache/kagglehub/datasets/imsparsh/fma-free-music-archive-small-medium/versions/1/fma_medium/fma_medium/080/080391.mp3: 
Progress: 23001/25000 files processed
Progress: 23501/25000 files processed


[src/libmpg123/parse.c:do_readahead():1083] warning: Cannot read next header, a one-frame stream? Duh...
/tmp/ipykernel_27187/2012216425.py:15: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(pathname, sr=conf.sampling_rate)
/home/rasmus/.local/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error processing /home/rasmus/.cache/kagglehub/datasets/imsparsh/fma-free-music-archive-small-medium/versions/1/fma_medium/fma_medium/126/126981.mp3: 
Progress: 24001/25000 files processed


[src/libmpg123/layer3.c:INT123_do_layer3():1844] error: dequantization failed!


Progress: 24501/25000 files processed


In [ ]:
#save_image_from_sound("000003_1.mp3")

/tmp/ipykernel_14529/1696889538.py:15: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(pathname, sr=conf.sampling_rate)
/home/rasmus/.local/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: '000003_1.mp3'